In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings
import glob

from skimage.filters import threshold_otsu
from skimage.feature import blob_dog
from pandas import read_csv
from skimage import io

plt.rcParams["figure.figsize"] = (20, 20)
plt.rcParams["image.cmap"] = "gray"
warnings.filterwarnings("ignore")

In [2]:
data = read_csv("./Data/BBBC021_v1_image.csv")

data.shape

(13200, 13)

In [3]:
data.iloc[0]

TableNumber                                                                     4
ImageNumber                                                                   233
Image_FileName_DAPI             G10_s1_w1BEDC2073-A983-4B98-95E9-84466707A25D.tif
Image_PathName_DAPI                                             Week4/Week4_27481
Image_FileName_Tubulin          G10_s1_w2DCEC82F3-05F7-4F2F-B779-C5DF9698141E.tif
Image_PathName_Tubulin                                          Week4/Week4_27481
Image_FileName_Actin            G10_s1_w43CD51CBC-2370-471F-BA01-EE250B14B3C8.tif
Image_PathName_Actin                                            Week4/Week4_27481
Image_Metadata_Plate_DAPI                                             Week4_27481
Image_Metadata_Well_DAPI                                                      G10
Replicate                                                                       1
Image_Metadata_Compound                                            5-fluorouracil
Image_Metadata_C

In [4]:
weeks = sorted(glob.glob("./DAPI/*/"))

In [5]:
patches = []
labels = []

for index, week in enumerate(weeks[: 1]):
    print(index, week)
    files = glob.glob(week + "*.jpeg")[: 2]
    print(len(files)).
    for file in files:
        #plt.figure(figsize = (20, 20));
        w = file.split("/")[-2]
        row = file.split("/")[-1].split(".")[0] + ".tif"
        #print(file)
        #print(w)
        #print(row)
        element = data[data["Image_FileName_DAPI"] == row].reset_index(drop = True)
        tubulin_path = "./Tubulin/" + w + "/" + element.loc[0, "Image_FileName_Tubulin"].split(".")[-2] + ".jpeg"
        actin_path = "./Actin/" + w + "/" + element.loc[0, "Image_FileName_Actin"].split(".")[-2] + ".jpeg"
        #plt.figure();
        image = io.imread(file)
        image_tubulin = io.imread(tubulin_path)
        image_actin = io.imread(actin_path)
        #_, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (20, 20));
        #axes[0].imshow(image);
        #axes[1].imshow(image_tubulin);
        #axes[2].imshow(image_actin);
        #plt.show()
        thresh = threshold_otsu(image)
        binary_image = (image > thresh)
        blobs = blob_dog(binary_image, min_sigma = 0.1, sigma_ratio = 5)
        for blob in blobs:
            y, x, r = blob
            if int(x) > 64 and int(y) > 64 and (int(x) < image.shape[1] - 64) and (int(y) < image.shape[0] - 64):
                x, y = int(x), int(y)
                if binary_image[y, x] == True:
                    #plt.imshow(image[y - 64: y + 64, x - 64: x + 64])
                    #plt.scatter(x, y, color = "red", marker = "+");
                    patch = image[y - 64: y + 64, x - 64: x + 64]
                    patch_tubulin = image_tubulin[y - 64: y + 64, x - 64: x + 64]
                    patch_actin = image_actin[y - 64: y + 64, x - 64: x + 64]
                    #_, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (20, 20));
                    #axes[0].imshow(patch);
                    #axes[1].imshow(patch_tubulin);
                    #axes[2].imshow(patch_actin);
                    #plt.show();
                    temp = np.stack((patch, patch_tubulin, patch_actin), axis = 2)
                    patches.append(temp)
                    labels.append(index)
        #plt.imshow(binary_image);
    
patches = np.stack(patches)
labels = np.array(labels)

0 ./DAPI/Week1/
2


In [6]:
patches.shape

(148, 128, 128, 3)

In [7]:
labels.shape

(148,)